<a href="https://colab.research.google.com/github/heejvely/third-project/blob/main/Etri_et5_train_60000_extra_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [1]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# module install 
!pip install transformers
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 3.1 MB 13.1 MB/s 
     |████████████████████████████████| 895 kB 56.3 MB/s 
     |████████████████████████████████| 3.3 MB 56.7 MB/s 
     |████████████████████████████████| 59 kB 7.6 MB/s 
     |████████████████████████████████| 596 kB 76.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 13.2 MB/s 


런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [3]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# model folder path
model_folder = '/content/drive/MyDrive/colab/3차/etri_et5'

model = T5ForConditionalGeneration.from_pretrained(model_folder)
tokenizer = T5Tokenizer.from_pretrained(model_folder)

In [4]:
# gpu로 지정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class CustomDataset:

# """
# author : Author of the article
# date : Date the article was published
# headline: Headline for the published article
# read_more : URL for the article to follow online
# text: This is the summary of the article
# ctext: This is the complete article
# """

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [6]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [7]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [8]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(45100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(45100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=3072, bias=False)
              (wi_1): Linear(in_features=768, out_features=3072, bias=False)
              (wo)

hyper-parameters

In [9]:
config = T5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 15       # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [10]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

In [11]:
!nvidia-smi

Thu Nov 25 06:51:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    39W / 300W |   3015MiB / 16160MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/train.csv')[['document','label']]
validation_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/valid.csv')[['document','label']]

partial dataset


In [13]:
import numpy as np

train_dataset = train_dataset.iloc[:30000]
validation_dataset = validation_dataset.iloc[:10000]

train

In [14]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


1


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  13.233085632324219


501it [02:43,  3.07it/s]

...

14001it [1:15:20,  3.10it/s]

Epoch: 14, Loss:  0.37435439229011536


14501it [1:18:01,  3.09it/s]

Epoch: 14, Loss:  0.6020517349243164


15000it [1:20:42,  3.10it/s]


In [15]:
# model save
tokenizer.save_pretrained('/content/drive/MyDrive/colab/3차/pretrained_train_30000_extra_training/')
model.save_pretrained('/content/drive/MyDrive/colab/3차/pretrained_train_30000_extra_training/')

valid(test)

In [16]:
validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Completed 1500
Completed 1600
Completed 1700
Completed 1800
Completed 1900
Completed 2000
Completed 2100
Completed 2200
Completed 2300
Completed 2400
Completed 2500
Completed 2600
Completed 2700
Completed 2800
Completed 2900
Completed 3000
Completed 3100
Completed 3200
Completed 3300
Completed 3400
Completed 3500
Completed 3600
Completed 3700
Completed 3800
Completed 3900
Completed 4000
Completed 4100
Completed 4200
Completed 4300
Completed 4400
Completed 4500
Completed 4600
Completed 4700
Completed 4800
Completed 4900


,Generated Text,Actual Text
0,2일 강다니엘 소속사 커넥트엔터테인먼트 측은 공식 홈페이지를 통해 9월 초 법률 자...,가수 강다니엘이 모욕죄 혐의로 악플러에 대한 고소장을 서울강남경찰서에 제출하며 합의...
1,를 위해 운영 중인 '2019년 안산시 청소년의회'가 의장단 구성을 마치고 본격적으...,안산시의회는 지난 7월 22일과 29일에 청소년의회 제1·2차 본회의를 열어 실제 ...
2,과 광산경찰서에 따르면 2019 광주세계수영선수권대회에서 여자 선수 12명의 신체를...,2019 광주세계수영선수권대회에서 18일 일본인A씨가 12명의 여자선수들의 신체 일...
3,"단지는 익산시 선화로53길 8 일원에 위치해있고, 지하1층, 지상최고 20층 3개 ...",제일건설은 실수요자들에게 선호도 높은 전용 면적과 효율적인 공간 활용을 강점으로 1...
4,"종목 등락률 1위를 차지한 에이치엘비는 등락률이 +14.87%이고, 2위는 한라IM...",월요일 11시 00분 현재 조선업종 내 상승종목 등락률 1위를 차지한 에이치엘비종목...
...,...,...
9995,순만복 조선대 교수의 ‘전 시민의 문화의병론’과 융진 인문통신이 발행하는 문화담론지...,"지역문화교류호남재단이 발행하는 문화담론지 ‘창’의 여름호(통권45호)가 나왔는데, ..."
9996,로 열린 일본에서 개막된 국제예술제인 '평화의 소녀상' 전시가 일본정부가 전시를 중...,일본에서 평화의 소녀상 전시가 지방자치단체 등의 단체가 전시를 중단하라 테러 협박 ...
9997,를 한국무역협회 국제무역연구원은 28일 삼성동 트레이드타워에서 열린 기자간담회에서 ...,한국 수출이 미.중 무역전쟁과 세계 보호무역 기조의 불확실성 및 제조업의 경쟁 심화...
9998,순조롭게 만들기 위해 지난 2016년 김승수 전주시장이 정치권과 정부에 필요성을 제...,전라선의 거점 정차역인데도 낡고 비좁았던 전주역이 지난 2016년 전주역사 신축사업...


In [17]:
final_df.to_csv('/content/drive/MyDrive/colab/3차/dataset/summarization_et5_train_30000_extra_training.csv')